In [1]:
import numpy as np
import astropy.units as u
import pandexo.engine.justdoit as jdi
import pandexo.engine.hst as hst
import matplotlib.pyplot as plt
import sys, os

from hst_functions import optimize_hst
from hst_functions import tweaked_pandexo

In [2]:
# First we started with just PandExo to estimate the RMS of the K1708 light curve
a, b = optimize_hst(jmag=14.429, hmag=14.168, transit_duration=19.13*u.hour)
print('Initial PandExo RMS Estimate: ' + str(a['light_curve_rms'] * 1e6))
print('Total transit depth uncertainty: ' + str(a['info']['Transit depth uncertainty(ppm)']))

****WARNING: Computed frame time (299 seconds) exceeds maximum recommended duration of 150 seconds.
Initial PandExo RMS Estimate: 349.7438850865515
Total transit depth uncertainty: 50.48118155053651


In [3]:
# But, as a sanity check, we used the same routine to get its estimate for Kepler-1625b
a, b = optimize_hst(jmag=14.364, hmag=13.989, transit_duration=18.821*u.hour)
print('PandExo estimate of K1625 RMS: ' + str(a['light_curve_rms'] * 1e6))
print('Total transit depth uncertainty: ' + str(a['info']['Transit depth uncertainty(ppm)']))

****WARNING: Computed frame time (299 seconds) exceeds maximum recommended duration of 150 seconds.
PandExo estimate of K1625 RMS: 339.42992645965955
Total transit depth uncertainty: 48.99248985312484


In [4]:
# That was far better than what was found in Teachey and Kipping 2018
#  (375.5ppm with their novel ramp correction, 440.1ppm without it)
# So, we started looking for other was to estimate.
# The core of PandExo's HST noise estimation is scaling from an observation of
#  GJ1214 (when using the G141 disperser, like we are).
# We copied the PandExo source code but modified it to allow for scaling based
#  on other reference targets. Here is the result using the exopential ramp
#  correction from Teachey and Kipping 2018
a, b = optimize_hst(jmag=14.429, hmag=14.168, transit_duration=19.13*u.hour)
c = tweaked_pandexo(refmag=14.364, refnoise=440.1, refexptime=298.8299)
d = c.wfc3_TExoNS(b)
print('PandExo but scaled from K1625 (exp ramp): '
      + str(d['light_curve_rms']*1e6))
print('Total transit depth uncertainty: '
          + str(d['info']['Transit depth uncertainty(ppm)']))

****WARNING: Computed frame time (299 seconds) exceeds maximum recommended duration of 150 seconds.
****WARNING: Computed frame time (299 seconds) exceeds maximum recommended duration of 150 seconds.
PandExo but scaled from K1625 (exp ramp): 453.4729316063552
Total transit depth uncertainty: 65.45317978328448


In [5]:
# Here is the same scheme but scaling from their more optimistic value
#  obtained via their non-parameterized ramp correction
a, b = optimize_hst(jmag=14.429, hmag=14.168, transit_duration=19.13*u.hour)
c = tweaked_pandexo(refmag=14.364, refnoise=375.5, refexptime=298.8299)
d = c.wfc3_TExoNS(b)
print('PandExo but scaled from K1625 (non-param ramp): '
      + str(d['light_curve_rms']*1e6))
print('Total transit depth uncertainty: '
          + str(d['info']['Transit depth uncertainty(ppm)']))

****WARNING: Computed frame time (299 seconds) exceeds maximum recommended duration of 150 seconds.
****WARNING: Computed frame time (299 seconds) exceeds maximum recommended duration of 150 seconds.
PandExo but scaled from K1625 (non-param ramp): 386.90998822582674
Total transit depth uncertainty: 55.84564646358401


In [6]:
# As a final sanity check, here is scaling purely off of the differences
#  in magnitude (should be same as modified PandExo)
old_flux = 1
flux_ratio = 10**((14.364 - 14.429)/2.5)
new_flux = flux_ratio * old_flux #* 19.44 / 18.821
new_variance = flux_ratio * (375.5e-6)**2 #* 19.44 / 18.821
print('Simple magnitude scaling: ' 
       + str(np.sqrt(new_variance)/new_flux * 1e6))

Simple magnitude scaling: 386.9099882258268
